In [2]:
%load_ext lab_black

In [21]:
import requests
import pandas as pd
from pandas import json_normalize
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import io

In [5]:
# Request data from the Edgar database

# SEC Edgar database API
# https://www.sec.gov/edgar/sec-api-documentation

In [6]:
def get_facts(cik: str, user_agent: str) -> dict:
    """A few quarters of financial reports."""
    assert len(cik) == 10
    url_facts = "https://data.sec.gov/api/xbrl/companyfacts/"
    headers = {"User-Agent": user_agent}
    resp = requests.get(
        url_facts + "CIK" + apple_cik + ".json", headers=headers, timeout=5
    )
    resp_json = None
    if resp.status_code < 400:
        resp_json = resp.json()
    return resp_json


def get_submissions(cik: str, user_agent: str) -> dict:
    """History of submissions of financial report fillings."""
    assert len(cik) == 10
    url_submissions = "https://data.sec.gov/submissions/"
    headers = {"User-Agent": user_agent}
    resp = requests.get(
        url_submissions + "CIK" + apple_cik + ".json", headers=headers, timeout=5
    )
    resp_json = None
    if resp.status_code < 400:
        resp_json = resp.json()
    return resp_json


def get_concepts(cik: str, entry: str, user_agent: str, taxonomy="us-gaap") -> dict:
    """hisotry of values for a specific financial report entry (e.g. Revenues)
    for a specific company.
    """
    assert len(cik) == 10
    url_concepts = "https://data.sec.gov/api/xbrl/companyconcepts/"
    headers = {"User-Agent": user_agent}
    resp = requests.get(
        url_concepts + "CIK" + cik + "/" + taxonomy + "/" + entry + ".json",
        headers=headers,
        timeout=5,
    )


def get_frames(
    cik: str,
    entry: str,
    period: str,
    currency: str,
    user_agent: str,
    taxonomy="us-gaap",
) -> dict:
    """"""
    assert len(cik) == 10
    url_frames = "https://data.sec.gov/api/xbrl/frames"
    headers = {"User-Agent": user_agent}
    resp = requests.get(
        f"{url_frames}/CIK{cik}/{taxonomy}/{entry}/{currency}/CY{period}.json",
        headers=headers,
        timeout=5,
    )

In [9]:
apple_cik = "0000320193"
user_agent = "Anselme F.E. Borgeaud (aborgeaud@gmail.com)"

In [14]:
subm_json = get_submissions(apple_cik, user_agent)

In [15]:
subm_json.keys()

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])

In [16]:
subm_json["formerNames"]

[{'name': 'APPLE INC',
  'from': '2007-01-10T00:00:00.000Z',
  'to': '2019-08-05T00:00:00.000Z'},
 {'name': 'APPLE COMPUTER INC',
  'from': '1994-01-26T00:00:00.000Z',
  'to': '2007-01-04T00:00:00.000Z'},
 {'name': 'APPLE COMPUTER INC/ FA',
  'from': '1997-07-28T00:00:00.000Z',
  'to': '1997-07-28T00:00:00.000Z'}]

In [20]:
# current list of all CIKs matched with entity name

resp = requests.get(
    "https://www.sec.gov/Archives/edgar/cik-lookup-data.txt",
    headers={"User-Agent": user_agent},
)
# all_current_ciks
resp_str = resp.content.decode("Latin-1")

'!J INC:0001438823:\n#1 A LIFESAFER HOLDINGS, INC.:0001509607:\n#1 ARIZONA DISCOUNT PROPERTIES LLC:0001'

In [45]:
lst = resp_str.split("\n")
lst2 = []
for i in range(len(lst)):
    lst2.append(lst[i].split(":")[:-1])

max_len = max([len(e) for e in lst2])

[e for e in lst2 if len(e) == 3][:10]

[['11', '11 CAPITAL CORP.', '0001463262'],
 ['29', '11 PLAN LLC', '0001738803'],
 ['2I CAPITAL PCC', ' STRATEGIC FUND I', '0001508295'],
 ['3', '10 CAPITAL WPF II, LLC', '0001728252'],
 ['3', '10 CAPITAL WPF III, LLC', '0001734857'],
 ['3', '10 CAPITAL WPF IV, LLC', '0001758234'],
 ['3', '10 CAPITAL WPF V, LLC', '0001776300'],
 ['5', '01 ACQUISITION CORP.', '0001823465'],
 ['5', '01 ACQUISITION LLC', '0001826636'],
 ['5', '15 FUND, LTD.', '0001464817']]

In [46]:
[line for line in lst if "10 CAPITAL WPF" in line]

['3:10 CAPITAL WPF II, LLC:0001728252:',
 '3:10 CAPITAL WPF III, LLC:0001734857:',
 '3:10 CAPITAL WPF IV, LLC:0001758234:',
 '3:10 CAPITAL WPF V, LLC:0001776300:']

In [22]:
file = io.StringIO(resp_str)
all_current_ciks = pd.read_csv(file, sep=':')

ParserError: Error tokenizing data. C error: Expected 3 fields in line 760, saw 4
